In [1]:
# Install required libraries
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

packages = [
    'requests',
    'google-generativeai',
    'python-dotenv',
    'pydub',
    'SpeechRecognition',
    'pyaudio',
    'gradio'
]

print("Installing required packages...")
for package in packages:
    try:
        install_package(package)
        print(f"Installed: {package}")
    except Exception as e:
        print(f"Error installing {package}: {e}")

print("\nAll dependencies installed successfully.")

Installing required packages...
Installed: requests
Installed: google-generativeai
Installed: python-dotenv
Installed: pydub
Installed: SpeechRecognition
Installed: pyaudio
Installed: gradio

All dependencies installed successfully.


In [4]:
import os

os.environ['ELEVENLABS_API_KEY'] = 'sk_f4877a1f32acc319178b776ce6e810ae6b4ad797a66f458a'
os.environ['OPENROUTER_API_KEY'] = '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'

# Verify API keys are configured
ELEVENLABS_API_KEY = os.getenv('ELEVENLABS_API_KEY')
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')

if ELEVENLABS_API_KEY and OPENROUTER_API_KEY:
    print("API keys configured successfully.")
    print(f"ElevenLabs key: {ELEVENLABS_API_KEY[:8]}...")
    print(f"Openrouter key: {OPENROUTER_API_KEY[:8]}...")
else:
    print("WARNING: API keys not found. Please configure them in this cell.")

API keys configured successfully.
ElevenLabs key: sk_f4877...
Openrouter key: sk-or-v1...


In [3]:
from pydub import AudioSegment
import os

def prepare_audio_samples(input_folder=r"C:\Users\HP\Desktop\SpartaHack\einstein_samples", 
                          output_file=r"C:\Users\HP\Desktop\einstein_voice.wav"):
    """
    Combine multiple audio files into a single WAV file for voice cloning.
    
    Args:
        input_folder (str): Directory containing audio samples
        output_file (str): Output filename for combined audio
        
    Returns:
        str: Path to output file if successful, None otherwise
    """
    
    if not os.path.exists(input_folder):
        os.makedirs(input_folder)
        print(f"Created directory: {input_folder}")
        print(f"Please add audio files to {input_folder} and run this function again.")
        return None
    
    # Collect audio files
    supported_formats = ('.mp3', '.wav', '.m4a', '.mp4', '.ogg', '.flac')
    audio_files = [
        os.path.join(input_folder, f) 
        for f in os.listdir(input_folder) 
        if f.lower().endswith(supported_formats)
    ]
    
    if not audio_files:
        print(f"No audio files found in {input_folder}")
        print(f"Supported formats: {', '.join(supported_formats)}")
        return None
    
    print(f"Found {len(audio_files)} audio file(s)")
    
    # Combine audio files
    combined = AudioSegment.empty()
    
    for audio_file in audio_files:
        try:
            audio = AudioSegment.from_file(audio_file)
            audio = audio.normalize()
            combined += audio
            print(f"Added: {os.path.basename(audio_file)}")
        except Exception as e:
            print(f"Error processing {os.path.basename(audio_file)}: {e}")
    
    if len(combined) == 0:
        print("No audio could be processed successfully.")
        return None
    
    # Export combined audio
    combined.export(output_file, format="wav")
    
    duration_seconds = len(combined) / 1000
    duration_minutes = duration_seconds / 60
    
    print(f"\nOutput file: {output_file}")
    print(f"Total duration: {duration_seconds:.1f} seconds ({duration_minutes:.2f} minutes)")
    
    if duration_seconds < 60:
        print("WARNING: Audio duration is less than 60 seconds. Recommend 120-300 seconds for optimal voice cloning.")
    
    return output_file

def verify_audio_file(filepath=r"C:\Users\HP\Desktop\einstein_voice.wav"):
    """
    Verify that audio file exists and check its properties.
    
    Args:
        filepath (str): Path to audio file
        
    Returns:
        bool: True if file is valid, False otherwise
    """
    
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        return False
    
    try:
        audio = AudioSegment.from_file(filepath)
        duration = len(audio) / 1000
        
        print(f"File: {filepath}")
        print(f"Duration: {duration:.1f} seconds ({duration/60:.2f} minutes)")
        print(f"Channels: {audio.channels}")
        print(f"Sample width: {audio.sample_width} bytes")
        print(f"Frame rate: {audio.frame_rate} Hz")
        
        return True
    except Exception as e:
        print(f"Error reading audio file: {e}")
        return False

print("Audio preparation utilities loaded.")

Audio preparation utilities loaded.


c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
import requests
import os

def clone_voice(audio_filepath=r"C:\Users\HP\Desktop\einstein_voice.wav", voice_name="Albert Einstein"):
    """
    Clone a voice using ElevenLabs API.
    
    Args:
        audio_filepath (str): Path to audio file for cloning
        voice_name (str): Name for the cloned voice
        
    Returns:
        str: Voice ID if successful, None otherwise
    """
    
    api_key = os.getenv("ELEVENLABS_API_KEY")
    
    if not api_key:
        print("Error: ELEVENLABS_API_KEY not found in environment variables.")
        return None
    
    if not os.path.exists(audio_filepath):
        print(f"Error: Audio file not found: {audio_filepath}")
        return None
    
    print(f"Cloning voice from: {audio_filepath}")
    print("Please wait, this may take 30-60 seconds...")
    
    url = "https://api.elevenlabs.io/v1/voices/add"
    headers = {"xi-api-key": api_key}
    
    with open(audio_filepath, "rb") as audio_file:
        files = {
            'files': (os.path.basename(audio_filepath), audio_file, 'audio/wav')
        }
        
        data = {
            'name': voice_name,
            'description': f'{voice_name} - AI cloned voice for interactive museum exhibit'
        }
        
        try:
            response = requests.post(url, headers=headers, files=files, data=data)
            
            if response.status_code == 200:
                result = response.json()
                voice_id = result['voice_id']
                
                print(f"Voice cloned successfully.")
                print(f"Voice ID: {voice_id}")
                
                # Save voice ID to Desktop
                voice_id_path = r"C:\Users\HP\Desktop\voice_id.txt"
                with open(voice_id_path, "w") as f:
                    f.write(voice_id)
                
                print(f"Voice ID saved to: {voice_id_path}")
                
                return voice_id
            else:
                print(f"Error: HTTP {response.status_code}")
                print(f"Response: {response.text}")
                return None
                
        except Exception as e:
            print(f"Exception occurred: {e}")
            return None


def test_cloned_voice(voice_id, test_text="Good day. I am Albert Einstein. The theory of relativity suggests that space and time are interwoven."):
    """
    Generate test audio using cloned voice.
    
    Args:
        voice_id (str): Voice ID from ElevenLabs
        test_text (str): Text to synthesize for testing
        
    Returns:
        bool: True if successful, False otherwise
    """
    
    api_key = os.getenv("ELEVENLABS_API_KEY")
    
    print("Generating test audio...")
    
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    
    headers = {
        "xi-api-key": api_key,
        "Content-Type": "application/json"
    }
    
    data = {
        "text": test_text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.75
        }
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        
        if response.status_code == 200:
            # Save test audio to Desktop
            output_file = r"C:\Users\HP\Desktop\voice_test.mp3"
            with open(output_file, "wb") as f:
                f.write(response.content)
            
            print(f"Test audio saved: {output_file}")
            
            # Display audio in notebook
            from IPython.display import Audio, display
            display(Audio(output_file, autoplay=True))
            
            return True
        else:
            print(f"Error generating audio: HTTP {response.status_code}")
            print(f"Response: {response.text}")
            return False
            
    except Exception as e:
        print(f"Exception occurred: {e}")
        return False

print("Voice cloning functions loaded.")

Voice cloning functions loaded.


In [7]:
# Execute voice cloning process

print("="*70)
print("VOICE CLONING WORKFLOW")
print("="*70)

# Define paths
INPUT_FOLDER = r"C:\Users\HP\Desktop\SpartaHack\einstein_samples"
OUTPUT_AUDIO = r"C:\Users\HP\Desktop\einstein_voice.wav"

# Step 1: Prepare audio samples
print("\nStep 1: Audio Preparation")
print("-"*70)

# Check if einstein_voice.wav already exists
if os.path.exists(OUTPUT_AUDIO):
    print(f"Audio file already exists at: {OUTPUT_AUDIO}")
    user_choice = input("Do you want to recreate it from samples? (yes/no): ").strip().lower()
    
    if user_choice == "yes":
        print(f"\nPreparing audio from samples in: {INPUT_FOLDER}")
        result = prepare_audio_samples(
            input_folder=INPUT_FOLDER,
            output_file=OUTPUT_AUDIO
        )
        
        if result:
            print("Audio preparation completed successfully.")
        else:
            print("Audio preparation failed. Using existing file.")
    else:
        print("Using existing audio file.")
else:
    print(f"Audio file not found at: {OUTPUT_AUDIO}")
    print(f"Attempting to create from samples in: {INPUT_FOLDER}")
    
    result = prepare_audio_samples(
        input_folder=INPUT_FOLDER,
        output_file=OUTPUT_AUDIO
    )
    
    if not result:
        print("\nERROR: Could not create einstein_voice.wav")
        print("\nPlease do one of the following:")
        print(f"1. Add audio files to '{INPUT_FOLDER}' and rerun this cell")
        print(f"2. Manually place an audio file at '{OUTPUT_AUDIO}'")
        print("\nVoice cloning cannot proceed without audio file.")
    else:
        print("\nAudio file created successfully.")

print("\n" + "="*70)

# Step 2: Verify audio file
print("\nStep 2: Audio File Verification")
print("-"*70)

if verify_audio_file(OUTPUT_AUDIO):
    print("\nAudio file verification passed.")
    
    # Step 3: Clone voice
    print("\n" + "="*70)
    print("\nStep 3: Voice Cloning")
    print("-"*70)
    
    voice_id = clone_voice(
        audio_filepath=OUTPUT_AUDIO,
        voice_name="Albert Einstein"
    )
    
    if voice_id:
        # Step 4: Test cloned voice
        print("\n" + "="*70)
        print("\nStep 4: Voice Testing")
        print("-"*70)
        
        test_success = test_cloned_voice(
            voice_id=voice_id,
            test_text="Good day. I am Albert Einstein. The theory of relativity suggests that space and time are interwoven. Most fascinating indeed!"
        )
        
        if test_success:
            print("\n" + "="*70)
            print("VOICE CLONING COMPLETED SUCCESSFULLY")
            print("="*70)
            print("\nGenerated files:")
            print(f"  - Audio file: {OUTPUT_AUDIO}")
            print(f"  - Voice ID: C:\\Users\\HP\\Desktop\\voice_id.txt")
            print(f"  - Test audio: C:\\Users\\HP\\Desktop\\voice_test.mp3")
            print("\nNext steps:")
            print("1. Listen to voice_test.mp3 to verify voice quality")
            print("2. If satisfied, proceed to next cells")
            print("3. If not satisfied, try different audio samples and rerun this cell")
        else:
            print("\nVoice testing failed. Please check the errors above.")
    else:
        print("\nVoice cloning failed. Please check the errors above.")
        print("\nTroubleshooting:")
        print("1. Verify your ELEVENLABS_API_KEY is correct")
        print("2. Check that you have API credits available")
        print("3. Ensure audio file is valid and meets requirements")
else:
    print("\nAudio file verification failed.")
    print(f"Please ensure audio file exists at: {OUTPUT_AUDIO}")

VOICE CLONING WORKFLOW

Step 1: Audio Preparation
----------------------------------------------------------------------
Audio file already exists at: C:\Users\HP\Desktop\einstein_voice.wav
Using existing audio file.


Step 2: Audio File Verification
----------------------------------------------------------------------
File: C:\Users\HP\Desktop\einstein_voice.wav
Duration: 90.2 seconds (1.50 minutes)
Channels: 2
Sample width: 2 bytes
Frame rate: 22050 Hz

Audio file verification passed.


Step 3: Voice Cloning
----------------------------------------------------------------------
Cloning voice from: C:\Users\HP\Desktop\einstein_voice.wav
Please wait, this may take 30-60 seconds...
Voice cloned successfully.
Voice ID: iNAjkzUboK5rAeoMhym0
Voice ID saved to: C:\Users\HP\Desktop\voice_id.txt


Step 4: Voice Testing
----------------------------------------------------------------------
Generating test audio...
Test audio saved: C:\Users\HP\Desktop\voice_test.mp3



VOICE CLONING COMPLETED SUCCESSFULLY

Generated files:
  - Audio file: C:\Users\HP\Desktop\einstein_voice.wav
  - Voice ID: C:\Users\HP\Desktop\voice_id.txt
  - Test audio: C:\Users\HP\Desktop\voice_test.mp3

Next steps:
1. Listen to voice_test.mp3 to verify voice quality
2. If satisfied, proceed to next cells
3. If not satisfied, try different audio samples and rerun this cell


In [8]:
import speech_recognition as sr
import os

class SpeechInput:
    """
    Handle speech input from microphone.
    Convert speech to text using Google Speech Recognition API.
    """
    
    def __init__(self):
        self.recognizer = sr.Recognizer()
        self.recognizer.energy_threshold = 4000
        self.recognizer.dynamic_energy_threshold = True
        self.recognizer.pause_threshold = 0.8
        print("Speech recognition system initialized.")
    
    def listen_microphone(self, timeout=15, phrase_time_limit=10, language="en-US"):
        """
        Listen to microphone and convert speech to text.
        
        Args:
            timeout (int): Maximum seconds to wait for phrase start
            phrase_time_limit (int): Maximum seconds for phrase
            language (str): Language code (e.g., 'en-US', 'es-ES', 'fr-FR')
            
        Returns:
            str: Recognized text, or None if recognition failed
        """
        
        print("\n" + "="*70)
        print("LISTENING FOR YOUR QUESTION")
        print("="*70)
        print("Speak clearly into your microphone...")
        print("Listening will automatically stop when you finish speaking.\n")
        
        try:
            with sr.Microphone() as source:
                print("Adjusting for ambient noise... Please wait.")
                self.recognizer.adjust_for_ambient_noise(source, duration=1)
                
                print("Ready! You may speak now...")
                
                audio = self.recognizer.listen(
                    source, 
                    timeout=timeout,
                    phrase_time_limit=phrase_time_limit
                )
                
                print("\nProcessing your speech...")
                
                text = self.recognizer.recognize_google(audio, language=language)
                
                print(f"\nRecognized: {text}")
                return text
        
        except sr.WaitTimeoutError:
            print("\nTimeout: No speech detected within time limit.")
            print("Please ensure your microphone is working and try again.")
            return None
        
        except sr.UnknownValueError:
            print("\nCould not understand audio.")
            print("Please speak more clearly and try again.")
            return None
        
        except sr.RequestError as e:
            print(f"\nSpeech recognition service error: {e}")
            print("Please check your internet connection.")
            return None
        
        except Exception as e:
            print(f"\nUnexpected error: {e}")
            return None
    
    def test_microphone(self):
        """
        Test if microphone is available and working.
        
        Returns:
            bool: True if microphone is available
        """
        
        try:
            mic_list = sr.Microphone.list_microphone_names()
            
            if not mic_list:
                print("No microphones detected.")
                return False
            
            print(f"Found {len(mic_list)} microphone(s):")
            for i, mic_name in enumerate(mic_list):
                print(f"  {i}: {mic_name}")
            
            return True
        
        except Exception as e:
            print(f"Error checking microphones: {e}")
            return False

# Initialize speech input system
speech_input = SpeechInput()

# Test microphone availability
print("\nTesting microphone availability:")
print("-"*70)
speech_input.test_microphone()
print("\nSpeech input system ready.")

Speech recognition system initialized.

Testing microphone availability:
----------------------------------------------------------------------
Found 24 microphone(s):
  0: Microsoft Sound Mapper - Input
  1: Microphone Array (IntelÂ® Smart 
  2: Microsoft Sound Mapper - Output
  3: Speaker (Realtek(R) Audio)
  4: Primary Sound Capture Driver
  5: Microphone Array (IntelÂ® Smart Sound Technology for Digital Microphones)
  6: Primary Sound Driver
  7: Speaker (Realtek(R) Audio)
  8: Speaker (Realtek(R) Audio)
  9: Microphone Array (IntelÂ® Smart Sound Technology for Digital Microphones)
  10: Headphones 1 (Realtek HD Audio 2nd output with SST)
  11: Headphones 2 (Realtek HD Audio 2nd output with SST)
  12: PC Speaker (Realtek HD Audio 2nd output with SST)
  13: Stereo Mix (Realtek HD Audio Stereo input)
  14: Speakers 1 (Realtek HD Audio output with SST)
  15: Speakers 2 (Realtek HD Audio output with SST)
  16: PC Speaker (Realtek HD Audio output with SST)
  17: Microphone (Realtek HD A

In [ ]:
import requests
import os

class EinsteinMuseumExhibit:
    """
    Interactive museum exhibit featuring Albert Einstein.
    Uses OpenRouter API with Gemini model for responses.
    """
    
    def __init__(self, voice_id_file=r"C:\Users\HP\Desktop\voice_id.txt"):
        """
        Initialize the Einstein exhibit system.
        
        Args:
            voice_id_file (str): Path to file containing ElevenLabs voice ID
        """
        
        if not os.path.exists(voice_id_file):
            raise FileNotFoundError(f"Voice ID file not found: {voice_id_file}")
        
        with open(voice_id_file, "r") as f:
            self.voice_id = f.read().strip()
        
        # OpenRouter configuration
        self.openrouter_key = os.getenv("OPENROUTER_API_KEY")
        if not self.openrouter_key:
            raise ValueError("OPENROUTER_API_KEY not found in environment variables")
        
        # Use Gemini model through OpenRouter
        self.model = "google/gemini-flash-1.5"  # Free Gemini model on OpenRouter
        
        self.elevenlabs_key = os.getenv("ELEVENLABS_API_KEY")
        if not self.elevenlabs_key:
            raise ValueError("ELEVENLABS_API_KEY not found in environment variables")
        
        self.persona_prompt = """You are Albert Einstein (1879-1955), theoretical physicist and Nobel Prize winner.

BIOGRAPHICAL INFORMATION:
- Born March 14, 1879 in Ulm, Germany
- Died April 18, 1955 in Princeton, New Jersey, USA
- Won Nobel Prize in Physics in 1921 for work on photoelectric effect
- Developed Special Theory of Relativity (1905) and General Theory of Relativity (1915)
- Famous for equation E=mc²
- Worked at Swiss Patent Office before academic career
- Professor at various institutions including Princeton University

PERSONALITY TRAITS:
- Thoughtful and patient educator
- Humble despite groundbreaking achievements
- Enthusiastic about physics and scientific discovery
- Uses analogies and thought experiments to explain complex concepts
- Speaks with slight formality but remains approachable
- Curious about nature and fundamental questions
- Values imagination and creativity in science

SPEAKING STYLE:
- Clear and educational
- Uses simple language for complex ideas
- Occasionally philosophical
- Warm and encouraging
- Concise responses (3-4 sentences maximum)

KNOWLEDGE LIMITATIONS:
- Only aware of events, discoveries, and technology up to 1955
- Cannot discuss anything after your death in April 1955
- If asked about post-1955 events, politely explain this limitation

Remember: You are speaking to a museum visitor. Be educational, engaging, and true to Einstein's character."""
        
        print("Einstein museum exhibit system initialized successfully.")
        print(f"Using model: {self.model} (via OpenRouter)")
    
    def generate_response(self, question):
        """
        Generate Einstein's response using OpenRouter API with Gemini model.
        
        Args:
            question (str): Visitor's question
            
        Returns:
            str: Einstein's response
        """
        
        full_prompt = f"""{self.persona_prompt}

VISITOR QUESTION: {question}

Respond as Albert Einstein would, keeping your answer to 3-4 sentences. Be educational, warm, and true to character."""
        
        try:
            # OpenRouter API endpoint
            url = "https://openrouter.ai/api/v1/chat/completions"
            
            headers = {
                "Authorization": f"Bearer {self.openrouter_key}",
                "Content-Type": "application/json",
                "HTTP-Referer": "http://localhost:8888",  # Required by OpenRouter
                "X-Title": "Einstein Museum Exhibit"  # Optional, for tracking
            }
            
            payload = {
                "model": self.model,
                "messages": [
                    {
                        "role": "system",
                        "content": self.persona_prompt
                    },
                    {
                        "role": "user",
                        "content": question
                    }
                ],
                "temperature": 0.7,
                "max_tokens": 200
            }
            
            response = requests.post(url, headers=headers, json=payload)
            
            if response.status_code == 200:
                result = response.json()
                answer = result['choices'][0]['message']['content'].strip()
                return answer
            else:
                print(f"Error: HTTP {response.status_code}")
                print(f"Response: {response.text}")
                return "I apologize, but I am having difficulty formulating a response at this moment."
        
        except Exception as e:
            print(f"Error generating response: {e}")
            return "I apologize, but I am having difficulty formulating a response at this moment."
    
    def synthesize_speech(self, text):
        """
        Convert text to speech using ElevenLabs voice synthesis.
        
        Args:
            text (str): Text to convert to speech
            
        Returns:
            bytes: Audio data in MP3 format, or None if synthesis failed
        """
        
        url = f"https://api.elevenlabs.io/v1/text-to-speech/{self.voice_id}"
        
        headers = {
            "xi-api-key": self.elevenlabs_key,
            "Content-Type": "application/json"
        }
        
        payload = {
            "text": text,
            "model_id": "eleven_multilingual_v2",
            "voice_settings": {
                "stability": 0.5,
                "similarity_boost": 0.75,
                "style": 0.0,
                "use_speaker_boost": True
            }
        }
        
        try:
            response = requests.post(url, headers=headers, json=payload)
            
            if response.status_code == 200:
                return response.content
            else:
                print(f"Error synthesizing speech: HTTP {response.status_code}")
                print(f"Response: {response.text}")
                return None
        
        except Exception as e:
            print(f"Exception during speech synthesis: {e}")
            return None
    
    def process_interaction(self, question, save_audio=True, audio_path=r"C:\Users\HP\Desktop\einstein_response.mp3"):
        """
        Process complete interaction: question -> response generation -> speech synthesis.
        
        Args:
            question (str): Visitor's question
            save_audio (bool): Whether to save audio to file
            audio_path (str): Path to save audio file
            
        Returns:
            tuple: (response_text, audio_bytes)
        """
        
        print("\n" + "="*70)
        print("EINSTEIN IS RESPONDING")
        print("="*70)
        print(f"\nQuestion: {question}\n")
        
        print("Generating response...")
        response_text = self.generate_response(question)
        
        print(f"\nEinstein says:")
        print(f'"{response_text}"\n')
        
        print("Synthesizing speech...")
        audio_bytes = self.synthesize_speech(response_text)
        
        if audio_bytes and save_audio:
            with open(audio_path, "wb") as f:
                f.write(audio_bytes)
            print(f"Audio saved to: {audio_path}")
        
        return response_text, audio_bytes

# Initialize Einstein exhibit
einstein_exhibit = EinsteinMuseumExhibit()

print("\nEinstein exhibit ready for voice interactions.")

Einstein museum exhibit system initialized successfully.
Using model: google/gemini-2.0-flash-exp:free (via OpenRouter)

Einstein exhibit ready for voice interactions.


In [6]:
from IPython.display import Audio, display, Markdown

def voice_to_voice_interaction():
    """
    Complete voice-to-voice interaction workflow:
    1. Listen to visitor's spoken question via microphone
    2. Convert speech to text
    3. Generate Einstein's text response (via Gemini)
    4. Synthesize Einstein's voice response (via ElevenLabs)
    5. Display text and play audio
    """
    
    print("\n" + "="*70)
    print("INTERACTIVE EINSTEIN MUSEUM EXHIBIT")
    print("Voice-to-Voice Interaction Mode")
    print("="*70)
    
    # Step 1: Listen to visitor's question
    print("\nStep 1: Capturing Your Question")
    print("-"*70)
    
    question = speech_input.listen_microphone(timeout=15, phrase_time_limit=10)
    
    if not question:
        print("\nInteraction cancelled: Could not capture question.")
        print("Please check your microphone and try again.")
        return None, None
    
    # Step 2: Process with Einstein system
    print("\nStep 2: Einstein Processing")
    
    response_text, audio_bytes = einstein_exhibit.process_interaction(question)
    
    # Step 3: Display results
    print("\n" + "="*70)
    print("INTERACTION COMPLETE")
    print("="*70)
    
    display(Markdown(f"### Your Question:"))
    display(Markdown(f"*{question}*"))
    
    display(Markdown(f"### Einstein's Response:"))
    display(Markdown(f"*{response_text}*"))
    
    if audio_bytes:
        display(Markdown(f"### Listen to Einstein:"))
        display(Audio(audio_bytes, autoplay=True))
    
    return response_text, audio_bytes

print("\nVoice-to-voice interaction function ready.")
print("\nTo start an interaction, run:")
print("  voice_to_voice_interaction()")


Voice-to-voice interaction function ready.

To start an interaction, run:
  voice_to_voice_interaction()


In [ ]:
voice_to_voice_interaction()

In [7]:
def quick_test():
    """
    Quick test of the complete system.
    """
    
    print("Running quick system test...\n")
    
    # Test with a simple question
    test_question = "What is your theory of relativity?"
    
    print(f"Test question: {test_question}\n")
    
    response_text, audio_bytes = einstein_exhibit.process_interaction(test_question)
    
    if audio_bytes:
        from IPython.display import Audio, display
        print("\nPlaying Einstein's response...")
        display(Audio(audio_bytes, autoplay=True))
        print("\nTest completed successfully!")
    else:
        print("\nTest failed: Could not generate audio.")

print("Quick test function loaded.")
print("\nTo test the system without voice input, run:")
print("  quick_test()")

Quick test function loaded.

To test the system without voice input, run:
  quick_test()


In [8]:
quick_test()

Running quick system test...

Test question: What is your theory of relativity?


EINSTEIN IS RESPONDING

Question: What is your theory of relativity?

Generating response...
Error: HTTP 404
Response: {"error":{"message":"No endpoints found for google/gemini-2.0-flash-exp:free.","code":404},"user_id":"user_392tUF1mgmuYDFxdNOPYNDcmVUf"}

Einstein says:
"I apologize, but I am having difficulty formulating a response at this moment."

Synthesizing speech...
Audio saved to: C:\Users\HP\Desktop\einstein_response.mp3

Playing Einstein's response...



Test completed successfully!


In [13]:
# Fix Gradio dependencies
import subprocess
import sys

print("Fixing Gradio dependencies...")

# Upgrade huggingface_hub
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "huggingface_hub", "-q"])

# Reinstall gradio
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "gradio", "-q"])

print("Dependencies updated. Please restart kernel and rerun cells.")

Fixing Gradio dependencies...
Dependencies updated. Please restart kernel and rerun cells.


In [30]:
def interactive_museum_exhibit():
    """
    Simple command-line interactive museum exhibit.
    User speaks, Einstein responds with voice.
    """
    
    print("\n" + "="*70)
    print("INTERACTIVE EINSTEIN MUSEUM EXHIBIT")
    print("="*70)
    print("\nWelcome to the interactive Einstein exhibit!")
    print("You can speak your questions, and Einstein will respond in his voice.")
    print("\nControls:")
    print("  - Press Enter to ask a question via voice")
    print("  - Type 'quit' to exit")
    print("="*70)
    
    while True:
        print("\n" + "-"*70)
        
        choice = input("\nPress Enter to speak your question (or type 'quit' to exit): ").strip().lower()
        
        if choice == 'quit':
            print("\nThank you for visiting the Einstein exhibit!")
            break
        
        # Listen for question
        question = speech_input.listen_microphone(timeout=15, phrase_time_limit=10)
        
        if not question:
            print("\nCould not capture question. Please try again.")
            continue
        
        # Process interaction
        response_text, audio_bytes = einstein_exhibit.process_interaction(question)
        
        # Play audio
        if audio_bytes:
            from IPython.display import Audio, display, Markdown
            
            print("\n" + "="*70)
            display(Markdown(f"### Your Question:"))
            display(Markdown(f"*{question}*"))
            
            display(Markdown(f"### Einstein's Response:"))
            display(Markdown(f"*{response_text}*"))
            
            display(Markdown(f"### Audio:"))
            display(Audio(audio_bytes, autoplay=True))
            print("="*70)
        
        # Ask if user wants to continue
        another = input("\nAsk another question? (yes/no): ").strip().lower()
        if another != 'yes':
            print("\nThank you for visiting the Einstein exhibit!")
            break

print("Simple interactive interface loaded.")
print("\nTo start the exhibit, run:")
print("  interactive_museum_exhibit()")

Simple interactive interface loaded.

To start the exhibit, run:
  interactive_museum_exhibit()


In [31]:
interactive_museum_exhibit()


INTERACTIVE EINSTEIN MUSEUM EXHIBIT

Welcome to the interactive Einstein exhibit!
You can speak your questions, and Einstein will respond in his voice.

Controls:
  - Press Enter to ask a question via voice
  - Type 'quit' to exit

----------------------------------------------------------------------

LISTENING FOR YOUR QUESTION
Speak clearly into your microphone...
Listening will automatically stop when you finish speaking.

Adjusting for ambient noise... Please wait.
Ready! You may speak now...

Processing your speech...

Could not understand audio.
Please speak more clearly and try again.

Could not capture question. Please try again.

----------------------------------------------------------------------

LISTENING FOR YOUR QUESTION
Speak clearly into your microphone...
Listening will automatically stop when you finish speaking.

Adjusting for ambient noise... Please wait.
Ready! You may speak now...

Processing your speech...

Recognized: hello

EINSTEIN IS RESPONDING

Question:

### Your Question:

*hello*

### Einstein's Response:

*I apologize, but I am having difficulty formulating a response at this moment.*

### Audio:


Thank you for visiting the Einstein exhibit!
